# Building AI Agents for Energy Sector Applications
## A Practical Workshop for Petroleum Engineers

This guide will walk you through building intelligent AI agents using LangGraph and AWS Amplify, with specific examples and applications relevant to the energy sector. As a petroleum engineer, you'll learn how these technologies can help you analyze data, optimize operations, and make better decisions.

## Prerequisites

Before beginning this workshop, make sure you have:
- An AWS account with appropriate permissions
- Node.js 18.x or later installed
- AWS CLI configured on your machine
- Basic understanding of JavaScript/TypeScript
- Familiarity with energy sector terminology and workflows

The workshop uses the following key packages:

In [1]:
require("esm-hook");

const { z } = require('zod');
const { tool } = require('@langchain/core/tools');

const { ChatBedrockConverse } = require("@langchain/aws");
const { HumanMessage, AIMessage, SystemMessage, BaseMessage } = require("@langchain/core/messages");
const { createReactAgent } = require("@langchain/langgraph/prebuilt");
const { Calculator } = require("@langchain/community/tools/calculator");
const { userInputTool } = require("../amplify/functions/tools/userInputTool.ts");
// const { pysparkTool } =   require("../amplify/functions/tools/athenaPySparkTool.ts");

const { 
    renderHumanMessage, 
    renderAIMessage, 
    renderUserInputToolMessage, 
    renderCalculatorToolMessage 
} = require('./helper_files/renderMessages.mjs');
const { displayAnimatedIndicator, invokeAgentAndRenderMessages, sampleAgent } = require('./helper_files/helperFunctions.mjs')

process.env.AWS_DEFAULT_REGION='us-east-1'

// Variables we'll use throughout the labs
let llm
let tools
let agent
let main
let myNewToolSchema
let myNewToolDefinition
let PermeabilityCalculatorSchema
let PermeabilityCalculator
let agentFinalState
let toolMessageResponse
let invokeAndRenderMessages


'us-east-1'

In [2]:
invokeAgentAndRenderMessages(
    `My naI need to buy a new downhole pump from Jeff for a Fruitland Coal well in New Mexico.
    The pump will be installed on the end of a tubing string with 100 joints of 30' pipe.
    Tell Jeff how deeply the pump will be installed.
    `,
    sampleAgent
)

⏳ Processing request... ........
✅ Response received successfully (took 9.0s)


User 
 I need to buy a new downhole pump from Jeff for a Fruitland Coal well in New Mexico.
 The pump will be installed on the end of a tubing string with 100 joints of 30' pipe.
 Tell Jeff how deeply the pump will be installed.
 
 


 
 Assistant 
 I'll help you calculate the total depth of the pump installation and send a message to Jeff with the details.

Let's calculate the total depth: 
 
 
 Tool Calls: 
 
 
 Tool: calculator 
 Arguments: {
 "input": "100 * 30"
} 
 
 
 
 
 


 
 Calculator 
 
 3000 
 
 


 
 Assistant 
 I'll now send a message to Jeff about the pump installation depth: 
 
 
 Tool Calls: 
 
 
 Tool: userInputTool 
 Arguments: {
 "title": "Downhole Pump Installation Details - Fruitland Coal Well, New Mexico",
 "description": "Jeff, \n\nI'm purchasing a new downhole pump for a well in the Fruitland Coal formation in New Mexico. \n\nPump Installation Details:\n- Total Tubing String Length: 100 joints of 30' pipe\n- Total Depth of Pump Installation: 3,000 feet\n\nPlease prepare the pump for installation at this depth.\n\nThank you,\n[Your Name]",
 "buttonTextBeforeClick": "Send Message to Jeff",
 "buttonTextAfterClick": "Message Sent"
} 
 
 
 
 
 


 
 Downhole Pump Installation Details - Fruitland Coal Well, New Mexico 
 Jeff, 

I'm purchasing a new downhole pump for a well in the Fruitland Coal formation in New Mexico. 

Pump Installation Details:
- Total Tubing String Length: 100 joints of 30' pipe
- Total Depth of Pump Installation: 3,000 feet

Please prepare the pump for installation at this depth.

Thank you,
[Your Name] 
 
 Send Message to Jeff
 
 
 

 
 Assistant 
 I've calculated the total depth of the pump installation and sent a message to Jeff. The calculation is straightforward:
- Number of pipe joints: 100
- Length of each joint: 30 feet
- Total depth: 100 × 30 = 3,000 feet

The message to Jeff includes the specific details about the pump installation depth and the total tubing string length. Is there anything else you would like me to do?

## Lab 1: Invoke Foundation Models from Amazon Bedrock in LangChain

In this lab, you'll learn how to initialize and interact with a large language model (Claude 3.5 Haiku) through Amazon Bedrock. This type of model can be used for analyzing geological reports, summarizing well performance data, or providing insights on reservoir management.

In [2]:
// Initialize Bedrock LLM
llm = new ChatBedrockConverse({
    model: "us.anthropic.claude-3-5-haiku-20241022-v1:0"
});

(async () => {
    const llmResponse = await displayAnimatedIndicator(
        llm.invoke("How can generative AI revolutionize the energy sector?")
    )
    console.log('llm Response:\n', llmResponse.content)
})()

⏳ Processing request... 

Promise { <pending> }

.......
✅ Response received successfully (took 7.1s)
llm Response:
 Generative AI can revolutionize the energy sector in several ways:

1. Grid Optimization
- Predictive maintenance
- Real-time energy demand forecasting
- Intelligent load balancing
- Enhanced grid stability management

2. Renewable Energy
- Optimizing solar and wind farm design
- Predicting renewable energy generation
- Improving energy storage technologies
- Modeling complex climate and generation patterns

3. Infrastructure Planning
- Simulating infrastructure scenarios
- Designing more efficient power plants
- Analyzing geographic potential for renewable projects
- Accelerating engineering and design processes

4. Energy Efficiency
- Developing advanced energy management systems
- Creating personalized energy consumption recommendations
- Identifying inefficiencies in industrial processes
- Optimizing building energy performance

5. Predictive Maintenance
- Detecting equipment failures before they occur
- Reducing d

**Foundation Model Invocation:** This simple setup allows you to query an AI model about any topic. The code initializes the Claude 3.5 Haiku model and sends a prompt to it, then displays the response.

In [6]:
(async () => {
    const llmResponse = await displayAnimatedIndicator(
        // llm.invoke(`What is the storage capacity of a reservoir with dimensions 854ft x 2458ft x 17.3ft and porosity 0.13?`)
        llm.invoke(`
        Calculate the net present value of a well with these charastics:
        - Current produciton of 1000 BOPD
        - 35% annual oil production decline rate
        - $50/BBL oil price
        - 15% annual PV discount rate
        - $250,000k / yr operating cost
    
        Respond with the result of the calculation
    
        Accronyms:
        - BOPD: Barrels of oil per day
    
        Formulas:
        - Economic Limit Production Rate (BOPD) = (Annual Operating Cost) / 365 / (Oil Price / BBL)
        - Economic Life Calculation (years) = log(<Economic Limit Production Rate>/<Current Production Rate>) / log(1 - <Annual Decline Rate>)

        `)
    )
    console.log('llm Response:\n', llmResponse.content)
})()

// Correct Answer: $35,123,843

⏳ Processing request... 

Promise { <pending> }

.....
✅ Response received successfully (took 5.0s)
llm Response:
 I'll solve this step by step:

1. Economic Limit Production Rate Calculation:
   • Annual Operating Cost = $250,000
   • Oil Price = $50/BBL
   • Economic Limit Rate = $250,000 / 365 / ($50/BBL)
   = 13.7 BOPD

2. Economic Life Calculation:
   • Current Production = 1000 BOPD
   • Decline Rate = 35%
   • Economic Life = log(13.7/1000) / log(1 - 0.35)
   = 3.4 years

3. NPV Calculation:
   • Annual Cash Flow = 1000 * 365 * $50 = $18,250,000 (Year 1)
   • Each subsequent year declines by 35%
   • Discount Rate = 15%

4. NPV Calculation (Simplified):
   Year 1: $18,250,000 * (1-0.35) / (1.15)¹
   Year 2: $18,250,000 * (1-0.35)² / (1.15)²
   Year 3: $18,250,000 * (1-0.35)³ / (1.15)³

5. Total NPV ≈ $32,500,000

The Net Present Value is approximately $32.5 million.


## Lab 2: Create Your First Agent

In this lab, you'll create an AI agent equipped with tools that can perform calculations and other operations. For petroleum engineers, this could involve calculating reservoir volumes, fluid properties, or economic metrics.


In [4]:
// Define available tools
tools = [
    new Calculator,
    // pysparkTool
];

// Create the React agent
agent = createReactAgent({
    llm,
    tools
});

CompiledStateGraph {
  lc_serializable: false,
  lc_kwargs: {
    checkpointer: undefined,
    interruptAfter: undefined,
    interruptBefore: undefined,
    autoValidate: false,
    nodes: {
      __start__: [PregelNode],
      agent: [PregelNode],
      tools: [PregelNode]
    },
    channels: {
      messages: [BinaryOperatorAggregate],
      structuredResponse: [LastValue],
      __start__: [EphemeralValue],
      agent: [EphemeralValue],
      tools: [EphemeralValue],
      'branch:__start__:__self__:agent': [EphemeralValue],
      'branch:__start__:__self__:tools': [EphemeralValue],
      'branch:agent:__self__:agent': [EphemeralValue],
      'branch:agent:__self__:tools': [EphemeralValue],
      'branch:tools:__self__:agent': [EphemeralValue],
      'branch:tools:__self__:tools': [EphemeralValue],
      '__start__:agent': [EphemeralValue],
      'branch:agent:condition:tools': [EphemeralValue]
    },
    inputChannels: '__start__',
    outputChannels: [ 'messages', 'structuredRe

**Energy Sector Application:** With a Calculator tool, your agent can perform basic arithmetic operations that could be useful in energy sector calculations. Note that this is a simple calculator that can only handle basic math operations (addition, subtraction, multiplication, division, etc.) - not complex petroleum engineering formulas directly.

For example, you could use it for:
- Simple components of reserve calculations
- Basic arithmetic in production analysis
- Elements of economic calculations
- Individual steps in engineering equations

Now you can test the agent with a calculation query:

In [7]:
invokeAgentAndRenderMessages(
    `Calculate the net present value of a well with these charastics:
    - Current produciton of 1000 BOPD
    - 35% annual oil production decline rate
    - $50/BBL oil price
    - 15% annual PV discount rate
    - $250,000k / yr operating cost

    Respond with the result of the calculation

    Accronyms:
    - BOPD: Barrels of oil per day

    Formulas:
    - Economic Limit Production Rate (BOPD) = (Annual Operating Cost) / 365 / (Oil Price / BBL)
    - Economic Life Calculation (years) = log(<Economic Limit Production Rate>/<Current Production Rate>) / log(1 - <Annual Decline Rate>)
    
    When using the calculator tool:
    - Use ^ for exponentials
    - There is no 'sum' function, intead use x+y+z to sum numbers.
    - Sum the discounted cash flows for multiple years in the same tool call
    `,
    agent
)
// Correct Answer: $35,123,843

⏳ Processing request... ...............
✅ Response received successfully (took 15.2s)


User 
 Calculate the net present value of a well with these charastics:
 - Current produciton of 1000 BOPD
 - 35% annual oil production decline rate
 - $50/BBL oil price
 - 15% annual PV discount rate
 - $250,000k / yr operating cost

 Respond with the result of the calculation

 Accronyms:
 - BOPD: Barrels of oil per day

 Formulas:
 - Economic Limit Production Rate (BOPD) = (Annual Operating Cost) / 365 / (Oil Price / BBL)
 - Economic Life Calculation (years) = log( / ) / log(1 - )
 
 When using the calculator tool:
 - Use ^ for exponentials
 - There is no 'sum' function, intead use x+y+z to sum numbers.
 - Sum the discounted cash flows for multiple years in the same tool call
 
 


 
 Assistant 
 I'll help you calculate the Net Present Value (NPV) for this well. I'll break this down step by step:

1. Economic Limit Production Rate Calculation: 
 
 
 Tool Calls: 
 
 
 Tool: calculator 
 Arguments: {
 "input": "250000 / 365 / 50"
} 
 
 
 
 
 

 Tool Message from calculator: 13.698630136986301 

 
 Assistant 
 Economic Limit Production Rate = 13.7 BOPD

2. Economic Life Calculation: 
 
 
 Tool Calls: 
 
 
 Tool: calculator 
 Arguments: {
 "input": "log(13.7/1000) / log(1 - 0.35)"
} 
 
 
 
 
 

 Tool Message from calculator: 9.959446593344627 

 
 Assistant 
 Economic Life = 10 years (rounding up)

3. NPV Calculation
I'll calculate the annual production and cash flow for each year, then discount it:

Year 1 Production: 1000 * (1 - 0.35) = 650 BOPD
Year 2 Production: 650 * (1 - 0.35) = 422.5 BOPD
And so on...

I'll calculate the cash flows (Revenue - OpEx) for each year and discount them: 
 
 
 Tool Calls: 
 
 
 Tool: calculator 
 Arguments: {
 "input": "(1000 * 365 * 50 - 250000) / (1 + 0.15)^1 + \n(1000 * (1 - 0.35) * 365 * 50 - 250000) / (1 + 0.15)^2 + \n(1000 * (1 - 0.35)^2 * 365 * 50 - 250000) / (1 + 0.15)^3 + \n(1000 * (1 - 0.35)^3 * 365 * 50 - 250000) / (1 + 0.15)^4 + \n(1000 * (1 - 0.35)^4 * 365 * 50 - 250000) / (1 + 0.15)^5 + \n(1000 * (1 - 0.35)^5 * 365 * 50 - 250000) / (1 + 0.15)^6 + \n(1000 * (1 - 0.35)^6 * 365 * 50 - 250000) / (1 + 0.15)^7 + \n(1000 * (1 - 0.35)^7 * 365 * 50 - 250000) / (1 + 0.15)^8 + \n(1000 * (1 - 0.35)^8 * 365 * 50 - 250000) / (1 + 0.15)^9 + \n(1000 * (1 - 0.35)^9 * 365 * 50 - 250000) / (1 + 0.15)^10"
} 
 
 
 
 
 

 Tool Message from calculator: 35123843.71814733 

 
 Assistant 
 Net Present Value (NPV) = $35,123,844

Key observations:
- Economic Limit Production Rate: 13.7 BOPD
- Economic Life: 10 years
- NPV: $35,123,844

The positive NPV indicates this is an economically viable project under the given assumptions.


This will produce an interactive result showing:
1. Your question
2. The AI's decision to use the calculator
3. The calculator's result (9417.689055530958)
4. The AI's final response with the formatted answer

**Energy Sector Application:** This calculation capability can be used for quick field calculations. For example, you could ask:
- "What is the storage capacity of a reservoir with dimensions 5000ft x 3000ft x 50ft and porosity 0.2?"
- "What flow rate is needed to achieve a 40% recovery factor in 10 years for a reservoir with 10 million barrels of OOIP?"


## Lab 3: Build Custom Tools

In this lab, you'll learn how to create custom tools tailored specifically for petroleum engineering applications.

In [27]:
//Define a ZOD object with the tool argument schema
myNewToolSchema = z.object({
    myFirstFunctionArgument: z.string(),
    changeArgumentNamesBasedOnUseCase: z.string(),
    useAsManyArgumentAsYouWant: z.string(),
    nestedArguments: z.object({
        areAllowed: z.string()
    })
})

myNewToolDefinition = tool(
    async (llmGeneratedArguments) => {
        console.log("my function code")
        return {
            success: true,
            ...llmGeneratedArguments
        }
    },
    {
        name: "replaceMeWithYourToolName",
        description: `
Update this description with 
`,
        schema: myNewToolSchema,
    }
);

DynamicStructuredTool {
  lc_serializable: false,
  lc_kwargs: {
    name: 'replaceMeWithYourToolName',
    description: '\nUpdate this description with \n',
    schema: ZodObject {
      spa: [Function: bound safeParseAsync] AsyncFunction,
      _def: [Object],
      parse: [Function: bound parse],
      safeParse: [Function: bound safeParse],
      parseAsync: [Function: bound parseAsync] AsyncFunction,
      safeParseAsync: [Function: bound safeParseAsync] AsyncFunction,
      refine: [Function: bound refine],
      refinement: [Function: bound refinement],
      superRefine: [Function: bound superRefine],
      optional: [Function: bound optional],
      nullable: [Function: bound nullable],
      nullish: [Function: bound nullish],
      array: [Function: bound array],
      promise: [Function: bound promise],
      or: [Function: bound or],
      and: [Function: bound and],
      transform: [Function: bound transform],
      brand: [Function: bound brand],
      default: [Functio

Let's put this custom tool to use:

In [29]:
// Define available tools
tools = [
    myNewToolDefinition
];

// Create the React agent
agent = createReactAgent({
    llm,
    tools,
});

invokeAgentAndRenderMessages(`Please call the tool`, agent)

⠙ Waiting for response... (1s)
⠹ Waiting for response... (2s)
⠸ Waiting for response... (3s)
my function code
⠼ Waiting for response... (4s)
⠴ Waiting for response... (5s)
⠦ Waiting for response... (6s)
Animation stopped - promise resolved


User 
 Please call the tool 
 


 
 Assistant 
 I'll help you call the tool. However, I notice that the available tool has a generic placeholder name and requires several mandatory arguments. I'll demonstrate calling the tool with some sample arguments: 
 

 Tool Message from replaceMeWithYourToolName: {
 "success": true,
 "myFirstFunctionArgument": "Sample First Argument",
 "changeArgumentNamesBasedOnUseCase": "Change Argument",
 "useAsManyArgumentAsYouWant": "Multiple Arguments",
 "nestedArguments": {
 "areAllowed": "Nested Argument Value"
 }
} 

 
 Assistant 
 I've called the tool with sample arguments that satisfy the required parameters:
1. `useAsManyArgumentAsYouWant`: A string value
2. `myFirstFunctionArgument`: A string value
3. `nestedArguments`: An object with the required "areAllowed" property
4. `changeArgumentNamesBasedOnUseCase`: A string value

The tool returned a success response with the arguments I provided. 

Is there a specific way you'd like me to use this tool or do you have specific values you'd like me to use for the arguments?

**Energy Sector Application:** Following this pattern, you can create specialized tools for petroleum engineering tasks, such as:

### Permeability Calculator Tool

In [31]:
PermeabilityCalculatorSchema = z.object({
    porosity: z.number().describe("Porosity (fraction)"),
    grainSize: z.number().describe("Average grain size (mm)"),
    rockType: z.enum(["sandstone", "limestone", "dolomite"]).describe("Type of reservoir rock")
});

PermeabilityCalculator = tool(
    async ({ porosity, grainSize, rockType }) => {
        // Simplified Kozeny-Carman equation
        let constant = 0;
        switch(rockType) {
          case "sandstone":
            constant = 150;
            break;
          case "limestone":
            constant = 225;
            break;
          case "dolomite":
            constant = 300;
            break;
        }
        
        // k = (porosity^3 * d^2) / (constant * (1-porosity)^2)
        const permeability = (Math.pow(porosity, 3) * Math.pow(grainSize, 2)) / 
                             (constant * Math.pow(1-porosity, 2));
        
        // Convert to millidarcy
        const permeabilityMD = permeability * 1000000;
        
        return {
          permeability_md: permeabilityMD.toFixed(2),
          rock_type: rockType,
          porosity: porosity,
          assessment: permeabilityMD > 100 ? "Good reservoir quality" : "Poor reservoir quality"
        }
    },
    {
      name: "permeability_calculator",
      description: "Calculate estimated permeability based on rock properties",
      schema: PermeabilityCalculatorSchema,
  }
);

// Define available tools
tools = [
    PermeabilityCalculator
];

// Create the React agent
agent = createReactAgent({
    llm,
    tools,
});

invokeAgentAndRenderMessages(`What is the permeability of 20% porosity sandstone with 1mm average grain size?`, agent)

⠙ Waiting for response... (1s)
⠹ Waiting for response... (2s)
⠸ Waiting for response... (3s)
⠼ Waiting for response... (4s)
⠴ Waiting for response... (5s)
Animation stopped - promise resolved


User 
 What is the permeability of 20% porosity sandstone with 1mm average grain size? 
 


 
 Assistant 
 I'll help you calculate the permeability of the sandstone with those characteristics using the permeability calculator tool. 
 

 Tool Message from permeability_calculator: {
 "permeability_md": "83.33",
 "rock_type": "sandstone",
 "porosity": 0.2,
 "assessment": "Poor reservoir quality"
} 

 
 Assistant 
 Let me break down the results for you:
- The calculated permeability is 83.33 millidarcies (md)
- The rock type is sandstone
- Porosity is 20% (0.2)
- The assessment suggests this is a "Poor reservoir quality" in terms of permeability

A few notes:
- Permeability measures how easily fluids can flow through the rock
- 83.33 md is considered relatively low permeability
- While the porosity is decent at 20%, the grain size and other factors limit the rock's ability to allow fluid flow
- This would likely be a challenging reservoir for oil or gas production without additional stimulation techniques

Would you like me to explain anything further about the permeability calculation?

## Lab 4: Custom Tool Response UI Elements

In this lab, you'll learn how to create custom UI elements for displaying tool responses in a more user-friendly way for petroleum engineers, such as charts and visualizations.

In [35]:
invokeAndRenderMessages = async (userInputText) => $$.html(
    (async () => {
        
        const result = await displayAnimatedIndicator(
            agent.invoke(
                { messages: [new HumanMessage(userInputText)] }
            )
        );
    
        // Render all messages in the conversation
        const conversationHtml = `
            <div style="font-family: system-ui, -apple-system, sans-serif; max-width: 800px; margin: 0 auto;">
                ${result.messages.map(message => {
                    switch(message.constructor.name) {
                        case 'HumanMessage':
                            return renderHumanMessage(message);
                        case 'AIMessage':
                            return renderAIMessage(message);
                        case 'ToolMessage':
                            switch (message.name) {
                                case 'calculator':
                                    return renderCalculatorToolMessage(message)
                                case 'userInputTool':
                                    return renderUserInputToolMessage(message);
                                default:
                                    return (`<div><h4>Tool Message from ${message.name}:</h4><pre>${message.content}</pre></div>`)
                            }
                            
                        default:
                            return (`<div><h4>Message:</h4><pre>${JSON.stringify(message, null, 2)}</pre></div>`);
                    }
                }).join('\n')}
            </div>
        `;
        
        return conversationHtml
    })()
)

// Define available tools
tools = [
    new Calculator,
    userInputTool
];

// Create the React agent
agent = createReactAgent({
    llm,
    tools,
});

invokeAndRenderMessages(
    `I need to buy a new downhole pump from Jeff for a Fruitland Coal well in New Mexico.
    The pump will be installed on the end of a tubing string with 100 joints of 30' pipe.
    Tell Jeff how deeply the pump will be installed.
    `
)

⠙ Waiting for response... (1s)
⠹ Waiting for response... (2s)
⠸ Waiting for response... (3s)
⠼ Waiting for response... (4s)
⠴ Waiting for response... (5s)
⠦ Waiting for response... (6s)
⠧ Waiting for response... (7s)
⠇ Waiting for response... (8s)
Animation stopped - promise resolved


User 
 I need to buy a new downhole pump from Jeff for a Fruitland Coal well in New Mexico.
 The pump will be installed on the end of a tubing string with 100 joints of 30' pipe.
 Tell Jeff how deeply the pump will be installed.
 
 

 Message from undefined: {
 "lc": 1,
 "type": "constructor",
 "id": [
 "langchain_core",
 "messages",
 "AIMessage"
 ],
 "kwargs": {
 "content": [
 {
 "type": "text",
 "text": "I'll help you calculate the total depth for the downhole pump installation and send a message to Jeff with the details.\n\nLet's calculate the total depth:"
 }
 ],
 "tool_calls": [
 {
 "id": "tooluse_60_UyTzLT8SUAgXNk9ZYkQ",
 "name": "calculator",
 "args": {
 "input": "100 * 30"
 },
 "type": "tool_call"
 }
 ],
 "response_metadata": {
 "$metadata": {
 "httpStatusCode": 200,
 "requestId": "a0773f99-3bb5-42e8-a697-2002d206731a",
 "attempts": 1,
 "totalRetryDelay": 0
 },
 "metrics": {
 "latencyMs": 1611
 },
 "stopReason": "tool_use",
 "usage": {
 "cacheReadInputTokenCount": 0,
 "cacheReadInputTokens": 0,
 "cacheWriteInputTokenCount": 0,
 "cacheWriteInputTokens": 0,
 "inputTokens": 615,
 "outputTokens": 88,
 "totalTokens": 703
 }
 },
 "usage_metadata": {
 "input_tokens": 615,
 "output_tokens": 88,
 "total_tokens": 703
 },
 "id": "a0773f99-3bb5-42e8-a697-2002d206731a",
 "invalid_tool_calls": [],
 "additional_kwargs": {}
 }
} 

 
 Calculator 
 
 3000 
 
 

 Message from undefined: {
 "lc": 1,
 "type": "constructor",
 "id": [
 "langchain_core",
 "messages",
 "AIMessage"
 ],
 "kwargs": {
 "content": [
 {
 "type": "text",
 "text": "Now, I'll send a message to Jeff with the installation depth information:"
 }
 ],
 "tool_calls": [
 {
 "id": "tooluse_7Y8GAJk5TZC4ytdn_WCY8A",
 "name": "userInputTool",
 "args": {
 "title": "Downhole Pump Installation Depth for Fruitland Coal Well",
 "description": "I need a downhole pump for a well in New Mexico with the following specifications:\n\n- Total tubing length: 100 joints of 30' pipe\n- Total installation depth: 3,000 feet\n\nPlease provide a pump suitable for this depth in the Fruitland Coal formation.",
 "buttonTextBeforeClick": "Send to Jeff",
 "buttonTextAfterClick": "Message Sent"
 },
 "type": "tool_call"
 }
 ],
 "response_metadata": {
 "$metadata": {
 "httpStatusCode": 200,
 "requestId": "9818e7d9-ee20-49ab-b4e1-52c95eb2559e",
 "attempts": 1,
 "totalRetryDelay": 0
 },
 "metrics": {
 "latencyMs": 4471
 },
 "stopReason": "tool_use",
 "usage": {
 "inputTokens": 716,
 "outputTokens": 209,
 "totalTokens": 925
 }
 },
 "usage_metadata": {
 "input_tokens": 716,
 "output_tokens": 209,
 "total_tokens": 925
 },
 "id": "9818e7d9-ee20-49ab-b4e1-52c95eb2559e",
 "invalid_tool_calls": [],
 "additional_kwargs": {}
 }
} 

 
 Downhole Pump Installation Depth for Fruitland Coal Well 
 I need a downhole pump for a well in New Mexico with the following specifications:

- Total tubing length: 100 joints of 30' pipe
- Total installation depth: 3,000 feet

Please provide a pump suitable for this depth in the Fruitland Coal formation. 
 
 Send to Jeff
 
 
 
 Message from undefined: {
 "lc": 1,
 "type": "constructor",
 "id": [
 "langchain_core",
 "messages",
 "AIMessage"
 ],
 "kwargs": {
 "content": "I've completed two tasks for you:\n1. Calculated the total installation depth by multiplying the number of pipe joints (100) by the length of each joint (30 feet), which comes to 3,000 feet.\n2. Sent a message to Jeff specifying the total installation depth and requesting a suitable downhole pump for the Fruitland Coal well in New Mexico.\n\nIs there anything else you would like me to do?",
 "response_metadata": {
 "$metadata": {
 "httpStatusCode": 200,
 "requestId": "ba5fd7e5-896e-4828-b278-dd40653e2003",
 "attempts": 1,
 "totalRetryDelay": 0
 },
 "metrics": {
 "latencyMs": 2322
 },
 "stopReason": "end_turn",
 "usage": {
 "cacheReadInputTokenCount": 0,
 "cacheReadInputTokens": 0,
 "cacheWriteInputTokenCount": 0,
 "cacheWriteInputTokens": 0,
 "inputTokens": 1063,
 "outputTokens": 99,
 "totalTokens": 1162
 }
 },
 "usage_

In this example, we've set up a more sophisticated UI that can handle:
1. The user's query about pump installation depth
2. The AI's response identifying the need to calculate depth
3. The calculator tool's computation (100 * 30 = 3000 feet)
4. A well-formatted message to "Jeff" with the equipment specifications
5. A button to "Send Details to Jeff" (interactive UI element)


## Lab 5: Store Conversation Messages Using AWS Amplify

In this lab, you'll learn how to persist conversation history using AWS Amplify and DynamoDB. This is valuable for maintaining context in long-running analyses or sharing insights across a team of engineers.

While this lab doesn't have concrete code examples in the notebook, the implementation would involve:

1. Setting up an Amplify API with a GraphQL schema
2. Creating a Conversation data type to store conversation history
3. Implementing functions to save and retrieve conversations